# Обучение модели Word2Vec на текстовых данных социальной сети VK, содержащих эмодзи

### Импорт необходимых библиотек

In [77]:
import pandas as pd
import nltk
from nltk.corpus import brown
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from gensim.models import Word2Vec as wv
from gensim.models import KeyedVectors
import pymorphy2
from pymorphy2 import MorphAnalyzer
import emoji
import zipfile
import numpy as np

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [5]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [7]:
nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_ru.zip.


True

In [26]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

### Подготовка датасета со смайликами

##### Чтение датасета 

In [33]:
emojis = pd.read_csv('marking.csv')

In [34]:
emojis = emojis.drop(columns=['image_id', 'name', 'group', 'sub_group', 'train'])

In [35]:
emojis

,text,emoji
0,Ухмыляющееся лицо,😀
1,Ухмыляющееся лицо с большими глазами,😃
2,Улыбающееся лицо с улыбающимися глазами,😄
3,Сияющее лицо с улыбающимися глазами,😁
4,Ухмыляющееся щурящееся лицо,😆
...,...,...
2744,Флаг: Замбия,🇲🇿
2745,Флаг: Зимбабве,🇿🇼
2746,Флаг: Англия,🏴󠁧󠁢󠁥󠁮󠁧󠁿
2747,Флаг: Шотландия,🏴󠁧󠁢󠁳󠁣󠁴󠁿


##### Токенизация описания эмодзи

In [36]:
regular = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
full_tokens = []
for text in emojis['text']:
  text = re.sub(regular, '', text)
  tokens = word_tokenize(text, language='russian')
  full_tokens.append(tokens)
emojis['tokens'] = full_tokens

In [37]:
emojis= emojis.drop('text', axis = 1)

In [38]:
emojis

,emoji,tokens
0,😀,"[Ухмыляющееся, лицо]"
1,😃,"[Ухмыляющееся, лицо, с, большими, глазами]"
2,😄,"[Улыбающееся, лицо, с, улыбающимися, глазами]"
3,😁,"[Сияющее, лицо, с, улыбающимися, глазами]"
4,😆,"[Ухмыляющееся, щурящееся, лицо]"
...,...,...
2744,🇲🇿,"[Флаг, Замбия]"
2745,🇿🇼,"[Флаг, Зимбабве]"
2746,🏴󠁧󠁢󠁥󠁮󠁧󠁿,"[Флаг, Англия]"
2747,🏴󠁧󠁢󠁳󠁣󠁴󠁿,"[Флаг, Шотландия]"


##### Приведение к нормальной форме всех слов в описании

In [39]:
filtered = []
morph = pymorphy2.MorphAnalyzer()
for tokens in emojis['tokens']:
  filtered_tokens = []
  for token in tokens:
    token = morph.normal_forms(token)[0]
    if token not in stopwords.words('russian'):
      filtered_tokens.append(token)
  filtered.append(filtered_tokens)

In [40]:
emojis['filtered'] = filtered

In [41]:
emojis = emojis.drop('tokens', axis = 1)

In [42]:
emojis

,emoji,filtered
0,😀,"[ухмыляться, лицо]"
1,😃,"[ухмыляться, лицо, больший, глаз]"
2,😄,"[улыбаться, лицо, улыбаться, глаз]"
3,😁,"[сиять, лицо, улыбаться, глаз]"
4,😆,"[ухмыляться, щуриться, лицо]"
...,...,...
2744,🇲🇿,"[флаг, замбия]"
2745,🇿🇼,"[флаг, зимбабве]"
2746,🏴󠁧󠁢󠁥󠁮󠁧󠁿,"[флаг, англия]"
2747,🏴󠁧󠁢󠁳󠁣󠁴󠁿,"[флаг, шотландия]"


##### Объединение смайлика с описанием для получения единого предложения для дальнейшего использования датасета для обучения модели

In [43]:
for e in emojis['emoji']:
  ind = emojis[emojis['emoji'] == e].index
  ind.to_list()
  emojis.at[ind[0], 'filtered'].append(e)

In [44]:
emojis = emojis.drop('emoji', axis = 1)

In [45]:
emojis

,filtered
0,"[ухмыляться, лицо, 😀]"
1,"[ухмыляться, лицо, больший, глаз, 😃]"
2,"[улыбаться, лицо, улыбаться, глаз, 😄]"
3,"[сиять, лицо, улыбаться, глаз, 😁]"
4,"[ухмыляться, щуриться, лицо, 😆]"
...,...
2744,"[флаг, замбия]"
2745,"[флаг, зимбабве, 🇿🇼]"
2746,"[флаг, англия, 🏴󠁧󠁢󠁥󠁮󠁧󠁿]"
2747,"[флаг, шотландия, 🏴󠁧󠁢󠁳󠁣󠁴󠁿]"


In [46]:
emojis['filtered'] = emojis.apply(lambda x: nltk.pos_tag(x['filtered'], tagset='universal', lang='rus'), axis=1) 

In [134]:
def tag_join(tag_list):
    joined_pares = []
    for pare in tag_list:
        if emoji.is_emoji(pare[0]):
            token_tag = pare[0] + '_SYM'
        else:
            token_tag = pare[0] + '_' + pare[1]
        joined_pares.append(token_tag)
    return joined_pares

In [48]:
emojis['u_tags'] = emojis.apply(lambda x: tag_join(x['filtered']), axis=1)

In [82]:
emojis

,filtered,u_tags
0,"[(ухмыляться, VERB), (лицо, NOUN), (😀, .)]","[ухмыляться_VERB, лицо_NOUN, 😀_SYM]"
1,"[(ухмыляться, VERB), (лицо, NOUN), (больший, A...","[ухмыляться_VERB, лицо_NOUN, больший_ADJ, глаз..."
2,"[(улыбаться, VERB), (лицо, NOUN), (улыбаться, ...","[улыбаться_VERB, лицо_NOUN, улыбаться_VERB, гл..."
3,"[(сиять, VERB), (лицо, NOUN), (улыбаться, VERB...","[сиять_VERB, лицо_NOUN, улыбаться_VERB, глаз_N..."
4,"[(ухмыляться, VERB), (щуриться, VERB), (лицо, ...","[ухмыляться_VERB, щуриться_VERB, лицо_NOUN, 😆_..."
...,...,...
2744,"[(флаг, NOUN), (замбия, NOUN)]","[флаг_NOUN, замбия_NOUN]"
2745,"[(флаг, NOUN), (зимбабве, VERB), (🇿🇼, .)]","[флаг_NOUN, зимбабве_VERB, 🇿🇼_SYM]"
2746,"[(флаг, NOUN), (англия, NOUN), (🏴󠁧󠁢󠁥󠁮󠁧󠁿, .)]","[флаг_NOUN, англия_NOUN, 🏴󠁧󠁢󠁥󠁮󠁧󠁿_SYM]"
2747,"[(флаг, NOUN), (шотландия, NOUN), (🏴󠁧󠁢󠁳󠁣󠁴󠁿, .)]","[флаг_NOUN, шотландия_NOUN, 🏴󠁧󠁢󠁳󠁣󠁴󠁿_SYM]"


### Подготовка датасета с комментариями ВК

##### Чтение датасета

In [182]:
colnames = ['index', 'comment']

In [183]:
comments = pd.read_csv('comments.csv', names=colnames, header=None)

In [184]:
comments

,index,comment
0,0,<class 'object'>
1,1,"Немцы так нехотя делают эти санкции, высылку д..."
2,2,а в чём прикол дипломатов туда-сюда гонять ?
3,3,"Вообще, это все логично. Абсолютно ожидаемое п..."
4,4,NaN
...,...,...
10100,17,"[id11875095|Сергей], даа, особенно после истор..."
10101,18,"[id155404424|Ruslan], про пытки в тюрьмах напо..."
10102,19,"[id11875095|Sergey], кто эти вы, которые туда ..."
10103,20,Это импорто замещения


In [185]:
comments = comments.drop(labels = [0,1],axis = 0)

In [186]:
comments = comments.drop(columns = ['index'],axis = 1)

In [187]:
comments = comments.reset_index(drop=True)

##### Предобработка комментариев

In [188]:
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

def lemmatize(doc):
    doc = re.sub(patterns, ' ', str(doc))
    for c in str(doc):
      if emoji.is_emoji(c):
          i = doc.index(c)
          doc = doc[:i] + doc[i:]
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru and emoji.is_emoji(token) == False:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            tokens.append(token)
        elif emoji.is_emoji(token):
            tokens.append(token)
    if len(tokens) > 2:
        return tokens
    return None

In [189]:
comments['comment'] = comments['comment'].apply(lemmatize)

In [192]:
comments = comments.dropna()

In [193]:
comments

,comment
0,"[что, прикол, дипломат, туда, сюда, гонять]"
1,"[вообще, это, логично, абсолютно, ожидать, пов..."
3,"[детишки, никто, изжалобить, весь, мир, против..."
4,"[складываться, ощущение, всё, отвечать, захаров]"
6,"[«никакий, правительство, мир, смочь, проводит..."
...,...
10098,"[сергей, даа, особенно, история, флойдом, прав..."
10099,"[пытка, тюрьма, напомнить, или, неудобно, гово..."
10100,"[который, туда, стремиться, любой, страна, ник..."
10101,"[это, импорто, замещение]"


In [194]:
comments['token_tag'] = comments.apply(lambda x: nltk.pos_tag(x['comment'], tagset='universal', lang='rus'), axis=1)

C:\Users\Admin\AppData\Local\Temp\ipykernel_10276\3793391976.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['token_tag'] = comments.apply(lambda x: nltk.pos_tag(x['comment'], tagset='universal', lang='rus'), axis=1)


In [202]:
comments

,comment,token_tag
0,"[что, прикол, дипломат, туда, сюда, гонять]","[что_CONJ, прикол_NOUN, дипломат_NOUN, туда_PR..."
1,"[вообще, это, логично, абсолютно, ожидать, пов...","[вообще_ADV, это_PRON, логично_ADJ, абсолютно_..."
3,"[детишки, никто, изжалобить, весь, мир, против...","[детишки_NOUN, никто_PRON, изжалобить_VERB, ве..."
4,"[складываться, ощущение, всё, отвечать, захаров]","[складываться_VERB, ощущение_NOUN, всё_PRON, о..."
6,"[«никакий, правительство, мир, смочь, проводит...","[«никакий_ADJ, правительство_NOUN, мир_NOUN, с..."
...,...,...
10098,"[сергей, даа, особенно, история, флойдом, прав...","[сергей_NOUN, даа_NOUN, особенно_ADV, история_..."
10099,"[пытка, тюрьма, напомнить, или, неудобно, гово...","[пытка_NOUN, тюрьма_NOUN, напомнить_VERB, или_..."
10100,"[который, туда, стремиться, любой, страна, ник...","[который_PRON, туда_PRON, стремиться_VERB, люб..."
10101,"[это, импорто, замещение]","[это_PRT, импорто_VERB, замещение_NOUN]"


In [201]:
comments['token_tag'] = comments.apply(lambda x: tag_join(x['token_tag']), axis=1)

C:\Users\Admin\AppData\Local\Temp\ipykernel_10276\1120379881.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['token_tag'] = comments.apply(lambda x: tag_join(x['token_tag']), axis=1)


##### Проверка на наличие эмодзи в полученном датасете комментариев

In [69]:
def check(comment):
  flag=0
  for item in comment:
    if emoji.is_emoji(item):
      flag+=1
  if flag > 0:
    return(comment)
  return None 

In [70]:
contain_emoji = comments.apply(check)

In [71]:
contain_emoji = contain_emoji.dropna()

In [72]:
contain_emoji

201      [алина, если, крестьянин, взять, рука, оружие,...
220      [никита, клин, клин, логика, разряд, мы, проти...
227      [анатолий, извинить, вопрос, данный, человек, ...
238      [фима, долго, искать, методичка, толстой, 😒, м...
316      [алина, если, крестьянин, взять, рука, оружие,...
                               ...                        
10038    [дмитрий, ахахахах, шок, 😂😂, сам, б, хотеть, н...
10049             [оля, ссать, речь, идти, 🤣, ещё, плохой]
10061                 [её, сосед, увидеть, весь, район, 😁]
10069    [адам, месяц, платить, налог, жизнь, нищеброд,...
10071     [анастасия, платить, быть, просмотр, готовый, 😄]
Name: comment, Length: 403, dtype: object

### Загрузка предобученной модели

##### Скачиваем обученную модель и загружаем её

In [222]:
model_1 = wv(vector_size=300, min_count=1, window=3)

In [223]:
model_1.build_vocab(comments['token_tag'])

In [224]:
model_1.wv.vectors_lockf = np.ones(len(model_1.wv))

In [225]:
model_1.wv.intersect_word2vec_format('model.bin', lockf=1.0, binary=True)

In [230]:
model_1.wv.most_similar('ощущение_NOUN')

[('чувство_NOUN', 0.6939337849617004),
 ('почувствовать_VERB', 0.5003801584243774),
 ('эмоция_NOUN', 0.45535600185394287),
 ('испытывать_VERB', 0.45355743169784546),
 ('мысль_NOUN', 0.44249626994132996),
 ('кайф_NOUN', 0.41636019945144653),
 ('дежа_NOUN', 0.4110955595970154),
 ('желание_NOUN', 0.3919849395751953),
 ('эмоциональный_ADJ', 0.3774087131023407),
 ('неприятный_ADJ', 0.36589956283569336)]

In [231]:
model_1.train(comments['token_tag'], total_examples=model_1.corpus_count, epochs=1)

(89372, 93291)

In [232]:
model_1.build_vocab(emojis['u_tags'], update=True)

In [233]:
model_1.wv.most_similar('😀_SYM')

[('🤱🏿_SYM', 0.22652676701545715),
 ('🕟_SYM', 0.20538826286792755),
 ('✍🏼_SYM', 0.1917581409215927),
 ('👶🏽_SYM', 0.1842544823884964),
 ('👩🏿\u200d🏫_SYM', 0.18080198764801025),
 ('🐟_SYM', 0.18076381087303162),
 ('🥬_SYM', 0.17668478190898895),
 ('🔜_SYM', 0.17590272426605225),
 ('👨🏾\u200d🔬_SYM', 0.17449399828910828),
 ('ванна_ADJ', 0.17345407605171204)]